# Renta Fija Local

En este notebook valorizaremos renta fija local utilizando una curva cupón cero, partir de eso calcularemos un spread y veremos como calcular la $\delta$ del bono respecto a la curva.

## Librerías

In [35]:
from datetime import date
import datetime
import bisect
import pandas as pd
import numpy as np

import autograd.numpy as agnp
from autograd import grad

import my_functions as mf

## Data de Bonos

Vamos a utilizar la misma base de datos de la sesión anterior.

In [2]:
bonos = pd.read_excel('data/bonos_empresa_carga_inicial.xlsx')

Al hacer el append, se regenera el índice. Con el valor por default `ignore_index=True` se matienen los índices originales de ambos `DataFrame`.

In [3]:
bonos.append(pd.read_excel('data/bonos_estado_carga_inicial.xlsx'), ignore_index=True)

,nemotecnico,fecha_emision,tasa_emision,tasa_descuento,unidad_monetaria,base_tasa_descuento,numero_flujos,meses_entre_vencimiento,tipo_intereses,numero_amortizaciones,plazo_agnos
0,BAARA-A,2017-12-01,3.85,3.8475,CLF,365,33,6,C,9,17
1,BAARA-B,2020-04-01,3.91,3.9075,CLF,365,28,6,C,13,14
2,BACEN-A1,2003-12-15,5.30,5.2960,CLF,365,46,6,C,34,23
3,BACEN-A2,2003-12-15,5.30,5.2960,CLF,365,46,6,C,34,23
4,BADAL-A,2017-12-01,3.85,3.8475,CLF,365,33,6,C,9,17
...,...,...,...,...,...,...,...,...,...,...,...
2091,PRC-7D0201,2001-02-01,6.50,6.5932,CLF,365,40,6,C,40,20
2092,PRC-7A0101,2001-01-01,6.50,6.5929,CLF,365,40,6,C,40,20
2093,PRC-7B0101,2001-01-01,6.50,6.5935,CLF,365,40,6,C,40,20
2094,PRC-7C0101,2001-01-01,6.50,6.5932,CLF,365,40,6,C,40,20


In [4]:
tablas_desarrollo = pd.read_csv('data/tablas_desarrollo.csv')

In [5]:
tablas_desarrollo.head()

,nemotecnico,numero_cupon,fecha_vcto_cupon,interes,amortizacion,saldo_insoluto
0,BC18-A0719,1,2019-10-31,0.98534,0.00,100.00
1,BC18-A0719,2,2020-01-31,0.98534,0.00,100.00
2,BC18-A0719,3,2020-04-30,0.98534,0.00,100.00
3,BC18-A0719,4,2020-07-31,0.98534,4.65,100.00
4,BC18-A0719,5,2020-10-31,0.93952,0.00,95.35


## Data de Curva Cero Cupón

Se utilizará la curva de gobierno en CLP para el 2021-08-20 construida por RiskAmerica. La tasa está en convención Comp Act/365.

In [6]:
curva_clp = pd.read_excel('data/20210820_curva_gob_ra.xlsx', sheet_name='curva_clp')

In [7]:
curva_clp.style.format({'tasa': '{:.4%}'})

,plazo,tasa
0,1,0.7999%
1,30,0.9300%
2,60,1.0598%
3,90,1.1849%
4,180,1.5333%
5,270,1.8455%
6,365,2.1403%
7,730,3.0141%
8,1095,3.6014%
9,1460,4.0117%


## Cálculo de Valor Presente con Curva Cero Cupón

Vamos a elegir bonos en CLP emitidos recientemente.

In [8]:
bonos[(bonos.unidad_monetaria=='CLP') & (bonos.fecha_emision >= '2020-01-01')]

,nemotecnico,fecha_emision,tasa_emision,tasa_descuento,unidad_monetaria,base_tasa_descuento,numero_flujos,meses_entre_vencimiento,tipo_intereses,numero_amortizaciones,plazo_agnos
282,BBNSAT0320,2020-03-01,3.00,3.0223,CLP,365,6,6,C,1,3
367,BCFSA-H,2020-05-15,3.60,3.5981,CLP,365,5,12,C,1,5
384,BCGEI-O,2020-06-20,3.20,3.1982,CLP,365,10,6,C,1,5
895,BFALA-AA,2020-04-15,3.45,3.4480,CLP,365,14,6,C,8,7
1184,BPARC-Z,2020-06-05,3.75,3.7478,CLP,365,10,6,C,1,5
1301,BSCCH-D,2020-03-01,3.50,3.4996,CLP,365,4,6,C,1,2
1302,BSCCH-E,2020-03-01,3.80,3.7997,CLP,365,6,6,C,1,3
1446,BSPED-E,2020-06-01,3.40,3.3981,CLP,365,10,6,C,1,5


De esta lista hay varios que no están colocados. Vamos a volver a utilizar el *BWATT-Q*.

In [9]:
tabla = tablas_desarrollo[tablas_desarrollo.nemotecnico=='BWATT-Q'].copy()

In [10]:
tabla.head()

,nemotecnico,numero_cupon,fecha_vcto_cupon,interes,amortizacion,saldo_insoluto
3277303,BWATT-Q,1,2020-03-01,1.9313,0.0,100.0
3277304,BWATT-Q,2,2020-09-01,1.9313,0.0,100.0
3277305,BWATT-Q,3,2021-03-01,1.9313,0.0,100.0
3277306,BWATT-Q,4,2021-09-01,1.9313,0.0,100.0
3277307,BWATT-Q,5,2022-03-01,1.9313,0.0,100.0


Para usar la curva cupón cero, tenemos que escribir una función que pueda calcular el valor presente a partir de ella.

In [29]:
def lin_interpol(plazo, plazos, tasas):
    """
    Implementa interpolación lineal.
    
    Parameters
    ----------
    
    plazo: float
        Plazo en días al que se quiere interpolar.
    
    plazos: array (numpy o autograd.numpy)
        Plazos en días de las tasas de la curva.
        
    tasas: array ((numpy o autograd.numpy))
        Tasas en en convención Comp Act/365 de la curva. Si se usa autograd, se puede derivar respecto
        a este parámetro.
        
    Returns
    -------
    
    Un `float` que corresponde al valor presente de los flujos.

    """
    i = bisect.bisect(plazos, plazo)
    if i == 0:
        return tasas[i]
    elif i == len(tasas):
        return tasas[i - 1]
    else:
        m = (tasas[i] - tasas[i - 1]) / (plazos[i] - plazos[i - 1])
        return tasas[i - 1] + m * (plazo - plazos[i - 1])

In [31]:
def valor_presente_curva(fecha_valor, fechas, flujos, plazos, tasas):
    """
    Calcula el valor presente de un conjunto de flujos utilizando una curva cero cupón. Para encontrar
    una tasa a un plazo específico se utiliza interpolación lineal.
    
    Parameters
    ----------
    
    fecha_valor: datetime.date
        Fecha a la cual se quiere obtener el valor presente.
        
    fechas: List[datetime.date]
        Fechas de pago de los flujos.
        
    flujos: List[float]
        Flujos a traer a valor presente. Deben corresponder a las fechas en el parámetro `fechas`.
        Los flujos cuyas fechas sean iguales o previas a `fecha_valor` no serán incluidos en el cálculo.
        
    plazos: numpy.array
        Plazos en días de las tasas de la curva.
        
    tasas: jnp.array
        Tasas en en convención Comp Act/365 de la curva.
        
    Returns
    -------
    
    Un `float` que corresponde al valor presente de los flujos.
    """
    result = 0.0
    for fec, fl in zip(fechas, flujos):
        p = (fec - fecha_valor).days
        if p > 0:
            tasa = lin_interpol(p, plazos, tasas)
            result += fl * agnp.power((1 + tasa), -p / 365.0)
    return result

Probemos ...

In [32]:
fechas = np.asarray([datetime.datetime.strptime(f, "%Y-%m-%d").date() for f in tabla['fecha_vcto_cupon'].values])

In [33]:
fecha_valor = date(2021, 8, 13)

In [36]:
resultado = valor_presente_curva(
    fecha_valor,
    fechas,
    tabla['interes'] + tabla['amortizacion'],
    curva_clp['plazo'],
    agnp.asarray(curva_clp['tasa'])
)
print(f'El valor presente es: {resultado:,.8f}')

El valor presente es: 96.12432231


¿Cómo podemos verificar el resultado? Hagamos un cálculo estilo Excel utilizando el `DataFrame` que contiene la tabla de desarrollo del bono.

Agregamos una columna `plazo`.

In [37]:
tabla['plazo'] = [(f - fecha_valor).days for f in fechas]

Agregamos una columna `tasa`.

In [38]:
tabla['tasa'] = agnp.interp(
    agnp.asarray(tabla['plazo']),
    curva_clp['plazo'],
    agnp.asarray(curva_clp['tasa']),
)

Agregamos la columna `df` que representa el factor de descuento asociado a la tasa.

In [39]:
tabla['df'] = np.power((1 + tabla['tasa']), -tabla['plazo'] / 365.0)

Agregamos la columna `vp` que es el valor presente del flujo.

In [40]:
tabla['vp'] = (tabla['interes'] + tabla['amortizacion']) * tabla['df']

Finalmente, se suman los valores presente de los flujos con `plazo` mayor a cero.

In [41]:
check = tabla[tabla.plazo > 0]['vp'].sum()
print(f'Check: {check:,.8f}')

Check: 96.12432231


## Spread Sobre Curva Cero Cupón

Notar que hemos valorizado un bono corporativo con una curva de bonos de gobierno. Esperaríamos que el valor presente así obtenido sea más alto que el que se obtendría descontando el mismo bono a su tir de mercado. Podemos calcular cuál es el spread implícito en la valorización con curva de gobierno. Para eso, se debe calcular cuál es la tir equivalente al descuento con curva. Dicho de otra forma, qué tir permite obtener el mismo valor presente que el calculado con la curva.

Para esto, vamos a reciclar y modificar una función que ya vimos al estudiar la instrucción `while`.

In [42]:
def encuentra_tir_2(fecha_valor, fechas, flujos, vp_obj, tasa):
    """
    Calcula la tasa de descuento o TIR que hace que el valor presente del bono (en base 100)
    sea igual a un valor dado.

    Parameters
    ----------

    fecha_valor: datetime.date
        Fecha a la cual se realiza el cálculo.

    plazos: List[float]
            Contiene las fechas de los flujos.

    flujos: List[float]
            Contiene los flujos del bono.

    vp_obj: float
        Valor presente del bono para el cual se quiere encontrar la tir.

    tasa: float
          Estimación inicial del valor del resultado.

    Returns
    -------

    float
         Tasa de descuento buscada.
    """
    epsilon = .000001
    diff = 1000
    der = grad(mf.valor_presente, 3)
    while diff > epsilon:
        q = ((mf.valor_presente(fecha_valor, fechas, flujos, tasa) - vp_obj) /
             der(fecha_valor, fechas, flujos, tasa)
             )
        nueva_tasa = tasa - q
        diff = abs(nueva_tasa - tasa)
        tasa = nueva_tasa
    return tasa

Notar que en esta versión de la función, se ha reemplazado el cálculo numérico de la derivada del bono respecto a su tir por el cálculo automático utilizando `autograd`.

In [43]:
tir_eq = encuentra_tir_2(
    fecha_valor,
    fechas,
    tabla['interes'] + tabla['amortizacion'],
    resultado,
    .02,
)
print(f'La TIR equivalente es: {tir_eq: .4%}')

La TIR equivalente es:  4.7593%


Podemos verificar el resultado:

In [44]:
check_tir_eq = mf.valor_presente(fecha_valor, fechas, tabla['interes'] + tabla['amortizacion'], tir_eq)
print(f'El valor presente con tir equivalente es: {check_tir_eq:,.6f}')

El valor presente con tir equivalente es: 96.124322


De acuerdo a RiskAmerica, el spread de este bono debiera ser 2.00%, o que la TIR base es 4.76%, que, a dos decimales es el mismo resultado que acabamos de obtener.

![RA](assets/20210820_bwatt-q_val_ra.png)

## Sensibilidad a Curva Cero Cupón

Se puede extender la idea de duración del bono (pensada como $\Delta$ del bono respecto a su tir de mercado) si consideramos el vector de $\delta$ del bono respecto a cada una de las tasas cupón cero. Usando `autograd` esto se calcula fácilmente de la siguiente forma:

In [45]:
# 4 corresponde a la posición de las tasas de la curva en la función valor_presente_curva.
gcurva = grad(valor_presente_curva, 4)

In [46]:
delta = gcurva(
    fecha_valor,
    fechas,
    tabla['interes'] + tabla['amortizacion'],
    agnp.asarray(curva_clp['plazo']),
    agnp.asarray(curva_clp['tasa'])
)

In [47]:
delta

array([-3.77832287e-02, -6.18271014e-02,  0.00000000e+00,  0.00000000e+00,
       -8.03070369e-01, -2.29448677e-01, -3.10775758e+00, -7.03290515e+00,
       -1.00265161e+01, -1.26720298e+01, -1.49749581e+01, -1.69493336e+01,
       -1.86678257e+01, -5.09503267e+02, -3.07401095e+01,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00])

Para calcular la sensibilidad a 1 punto básico en cada vértice basta con lo siguiente (para ver cifras significativas vamos a asumir un nominal de 100,000,000 CLP):

In [55]:
nominal = 100000000
sens = [d * .0001 * nominal for d in delta]
for i, s in enumerate(sens):
    print(f'Sensibilidad vértice {i} al plazo {curva_clp.iloc[i, 0]}: {s:,.0f}')

Sensibilidad vértice 0 al plazo 1: -378
Sensibilidad vértice 1 al plazo 30: -618
Sensibilidad vértice 2 al plazo 60: 0
Sensibilidad vértice 3 al plazo 90: 0
Sensibilidad vértice 4 al plazo 180: -8,031
Sensibilidad vértice 5 al plazo 270: -2,294
Sensibilidad vértice 6 al plazo 365: -31,078
Sensibilidad vértice 7 al plazo 730: -70,329
Sensibilidad vértice 8 al plazo 1095: -100,265
Sensibilidad vértice 9 al plazo 1460: -126,720
Sensibilidad vértice 10 al plazo 1825: -149,750
Sensibilidad vértice 11 al plazo 2190: -169,493
Sensibilidad vértice 12 al plazo 2555: -186,678
Sensibilidad vértice 13 al plazo 2920: -5,095,033
Sensibilidad vértice 14 al plazo 3285: -307,401
Sensibilidad vértice 15 al plazo 3650: 0
Sensibilidad vértice 16 al plazo 4380: 0
Sensibilidad vértice 17 al plazo 5475: 0
Sensibilidad vértice 18 al plazo 7300: 0


### Ejercicio

Verificar el resultado anterior de forma numérica.